In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pylab import plot,show,hist
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm, chi2_contingency

#Настройки показа графиков в блокноте
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.width', 85)
pd.set_option('display.max_columns', 8)

from sklearn import preprocessing
#%config InlineBackend.figure_format = 'svg' для большей четкости графиков
matplotlib.style.use('ggplot')
%matplotlib inline

df = pd.read_csv('marketing_data.csv')

In [19]:
df.head().T

,0,1,2,3,4
ID,1826,1,10476,1386,5371
Year_Birth,1970,1961,1958,1967,1989
Education,Graduation,Graduation,Graduation,Graduation,Graduation
Marital_Status,Divorced,Single,Married,Together,Single
Income,"$84,835.00","$57,091.00","$67,267.00","$32,474.00","$21,474.00"
Kidhome,0,0,0,1,1
Teenhome,0,0,1,1,0
Dt_Customer,6/16/14,6/15/14,5/13/14,5/11/14,4/8/14
Recency,0,0,0,0,0
MntWines,189,464,134,10,6


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4    Income              2216 non-null   object 
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [17]:
df['Income'] = df[' Income '].str.replace(r'\D+', '')
df['Income'] = pd.to_numeric(df['Income'], errors='coerce')

In [ ]:
#Удаляем столбцы, где данные непрочитались
df = df.filter(regex='^(?!.*Unnamed).*$')

In [ ]:
#Смотрим коэффициенты корелляций. Мало больших значений - плохо для факторного анализа
df.corr()

In [ ]:
#Сортировка числовых колонок
df['data'].value_counts(normalize=True)

In [ ]:
print('Текущая длина фрейма: ', len(df))

In [ ]:
#Чистка от пустых значений
df = df.replace(-9999, np.nan)
print('Rows in the data frame: {0}'.format(len(df)))
print('Rows without NAN: {0}'.format(len(df.dropna(how='any'))))

In [ ]:
#Слишком много данных содержат хотя бы одно пропущенное значение, чтобы удалить их все. Смотрим их распределение по колонкам
#Функция .apply для всей матрицы. 1й аргумент-применяемая функция, 2й - направление применения (0 к колонкам, 1 ко строкам)
df.apply(lambda x: sum(x.isnull()), axis=0)

In [ ]:
#Анализируем колонку где можно заменить пропуски
df['data'].hist()

In [ ]:
#Меняем пропущенные значения на среднее значение по колонке 
df['data'] = df['data'].fillna(df['data'].mean())

In [ ]:
#Удалим самые пустые колонки полностью и выбросим строчки с пропущенными значениями в тех где нужно
df = df.drop(['data', 'data'], axis=1)
df = df[(df['data'] != 'unknown') & (df['data'] != 'unknown')]

In [ ]:
#Получение сокращенного фрейма по конкретным критериям
df[df["Sex"].isin([True])].head()

In [ ]:
# Преобразуем строчки с датами в объект datetime
df['date'] = pd.to_datetime(df['date'], format='%b %Y') # format показывает что читаем: '%b %Y' трехбуквенный месяц, затем год 

In [ ]:
#Построим график проверить тип тренда (линейный или нет), тип сезонности (аддитивный или мультипликативный), его длину, выбросы
#Видим линейный тренд и мультипликативную сезонность. Это подтверждается после логирафмирование цикла 

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
df['date'].plot(ax=ax1)
ax1.set_title(u'коммент')
ax1.set_ylabel(u'коммент')

ax2 = fig.add_subplot(122)
pd.Series(np.log10(df['date'])).plot(ax=ax2)
ax2.set_title(u'коммент')
ax2.set_ylabel(u'коммент')

In [ ]:
#Shapiro-Wilk позволяют проверить выборку на принадлежность к ГС и нормальность распредеелния
df = df.set_index(u'номер')
plt.hist(np.log10(df[u'номер']), bins=50) #Применяем критерий Шапиро-Вилка после логарифмирования
res = stats.shapiro(np.log10(df[u'номер']))
print('p-value: ', res[1]) #Если p маленькое, гипотезу о нормальности отвергаем

In [ ]:
#Работа с выбросами
town_2 = df.iloc[2:1004] #Обрезать аномалии

x = np.log10(df[u'номер']) #Логарифмировать переменную (для лог-нормального распределения)
pd.Series(x).hist(bins=45)

exclude = int(len(df)/100*2.5) #Усеченное среднее (Среднее -2,5% самых малых и -2,5% наибольших)
redacted_town = df[exclude:len(df)-exclude]

In [ ]:
#Корреляция двух переменных
plt.scatter(df['data'], df['data'])

In [ ]:
#Исправление опечаток
df['data'].map({'m': 'male', fem.': 'female', ...})
re.sub (r "\ ^ m \ $", ' Male ', ' male ', flags = re. IGNORECASE )

In [ ]:
#Заполнить пробелы Средним значением
rand = np.random.randint(average_age - 2*std_age, average_age + 2*std_age, size = count_nan_age) 
df["data"][np.isnan(df["data"])] = rand

In [ ]:
#Строим R матрицу корелляций. Много выбросов, есть бимодальности. Но сильной корелляции увы нет
scatter_matrix(df);